# 第十四章 惡龍寶區：Reflect、Unsafe 與 Cgo

## 章節概述

本章探討 Go 語言中三個強大但需要謹慎使用的特性：反射（Reflection）、不安全操作（Unsafe）和 C 語言互操作（Cgo）。這些特性被稱為「惡龍寶區」，因為它們提供了巨大的力量，但也帶來了相應的風險和複雜性。

### 本章重點內容
- **反射 (Reflection)**: 在執行時檢查和操作型別資訊
- **Type、Kind 與 Value**: 反射的核心概念
- **反射的實際應用**: JSON 序列化、ORM 框架等
- **unsafe 套件**: 繞過 Go 的型別安全
- **記憶體操作**: 直接操作記憶體位址
- **Cgo**: 與 C 程式庫的互操作
- **效能考量**: 何時使用和避免這些特性

### 使用原則
1. **最後手段**: 只有在必要時才使用
2. **效能導向**: 主要用於效能關鍵的場景
3. **庫作者**: 通常由庫作者使用，應用開發者較少直接使用
4. **謹慎測試**: 需要更多的測試和文檔

In [ ]:
// 反射、Unsafe 和 Cgo 示範
package main

import (
    "fmt"
    "reflect"
    "unsafe"
    "encoding/json"
    "strings"
)

// 測試用的結構
type Person struct {
    Name    string `json:"name" validate:"required"`
    Age     int    `json:"age" validate:"min=0,max=150"`
    Email   string `json:"email" validate:"email"`
    private string // 私有欄位
}

// Product 結構用於示範反射
type Product struct {
    ID          int     `json:"id"`
    Name        string  `json:"name"`
    Price       float64 `json:"price"`
    InStock     bool    `json:"in_stock"`
    Tags        []string `json:"tags"`
}

func main() {
    fmt.Println("=== 惡龍寶區：Reflection, Unsafe, Cgo ===")
    
    // 1. 反射基礎
    fmt.Println("\n1. 反射基礎:")
    demonstrateReflectionBasics()
    
    // 2. 反射的實際應用
    fmt.Println("\n2. 反射實際應用:")
    demonstrateReflectionApplications()
    
    // 3. Unsafe 套件
    fmt.Println("\n3. Unsafe 套件:")
    demonstrateUnsafe()
    
    // 4. 效能比較
    fmt.Println("\n4. 效能比較:")
    demonstratePerformanceComparison()
}

// 示範反射基礎
func demonstrateReflectionBasics() {
    person := Person{
        Name:    "張三",
        Age:     30,
        Email:   "zhang@example.com",
        private: "secret",
    }
    
    // 獲取反射值
    v := reflect.ValueOf(person)
    t := reflect.TypeOf(person)
    
    fmt.Printf("  型別名稱: %s\n", t.Name())
    fmt.Printf("  型別種類: %s\n", t.Kind())
    fmt.Printf("  欄位數量: %d\n", t.NumField())
    
    // 遍歷結構欄位
    fmt.Println("  結構欄位:")
    for i := 0; i < t.NumField(); i++ {
        field := t.Field(i)
        fieldValue := v.Field(i)
        
        fmt.Printf("    %s: %v (型別: %s)\n", 
            field.Name, fieldValue.Interface(), field.Type)
        
        // 檢查結構標籤
        if jsonTag := field.Tag.Get("json"); jsonTag != "" {
            fmt.Printf("      JSON 標籤: %s\n", jsonTag)
        }
        if validateTag := field.Tag.Get("validate"); validateTag != "" {
            fmt.Printf("      驗證標籤: %s\n", validateTag)
        }
    }
    
    // 檢查不同型別的 Kind
    fmt.Println("\n  不同型別的 Kind:")
    values := []interface{}{
        42,
        "hello",
        []int{1, 2, 3},
        map[string]int{"a": 1},
        &person,
    }
    
    for _, val := range values {
        v := reflect.ValueOf(val)
        fmt.Printf("    %T -> Kind: %s\n", val, v.Kind())
    }
}

// 示範反射的實際應用
func demonstrateReflectionApplications() {
    // 1. 動態 JSON 序列化
    fmt.Println("  1. 動態 JSON 序列化:")
    
    product := Product{
        ID:      1,
        Name:    "筆記型電腦",
        Price:   29999.99,
        InStock: true,
        Tags:    []string{"電子", "電腦", "辦公"},
    }
    
    jsonStr := structToJSON(product)
    fmt.Printf("    動態序列化結果: %s\n", jsonStr)
    
    // 2. 結構驗證
    fmt.Println("\n  2. 結構驗證:")
    
    validPerson := Person{Name: "李四", Age: 25, Email: "li@example.com"}
    invalidPerson := Person{Name: "", Age: -5, Email: "invalid-email"}
    
    fmt.Printf("    有效人員驗證: %v\n", validateStruct(validPerson))
    fmt.Printf("    無效人員驗證: %v\n", validateStruct(invalidPerson))
    
    // 3. 動態方法調用
    fmt.Println("\n  3. 動態方法調用:")
    demonstrateMethodInvocation()
}

// 使用反射將結構轉換為 JSON（簡化版）
func structToJSON(v interface{}) string {
    val := reflect.ValueOf(v)
    typ := reflect.TypeOf(v)
    
    if val.Kind() != reflect.Struct {
        return fmt.Sprintf(`"%v"`, v)
    }
    
    var parts []string
    
    for i := 0; i < val.NumField(); i++ {
        field := typ.Field(i)
        fieldValue := val.Field(i)
        
        // 跳過私有欄位
        if !fieldValue.CanInterface() {
            continue
        }
        
        jsonTag := field.Tag.Get("json")
        if jsonTag == "" {
            jsonTag = strings.ToLower(field.Name)
        }
        
        var valueStr string
        switch fieldValue.Kind() {
        case reflect.String:
            valueStr = fmt.Sprintf(`"%s"`, fieldValue.String())
        case reflect.Int, reflect.Int8, reflect.Int16, reflect.Int32, reflect.Int64:
            valueStr = fmt.Sprintf("%d", fieldValue.Int())
        case reflect.Float32, reflect.Float64:
            valueStr = fmt.Sprintf("%.2f", fieldValue.Float())
        case reflect.Bool:
            valueStr = fmt.Sprintf("%t", fieldValue.Bool())
        case reflect.Slice:
            var items []string
            for j := 0; j < fieldValue.Len(); j++ {
                item := fieldValue.Index(j)
                items = append(items, fmt.Sprintf(`"%v"`, item.Interface()))
            }
            valueStr = fmt.Sprintf("[%s]", strings.Join(items, ","))
        default:
            valueStr = fmt.Sprintf(`"%v"`, fieldValue.Interface())
        }
        
        parts = append(parts, fmt.Sprintf(`"%s":%s`, jsonTag, valueStr))
    }
    
    return fmt.Sprintf("{%s}", strings.Join(parts, ","))
}

// 簡單的結構驗證
func validateStruct(v interface{}) []string {
    var errors []string
    
    val := reflect.ValueOf(v)
    typ := reflect.TypeOf(v)
    
    for i := 0; i < val.NumField(); i++ {
        field := typ.Field(i)
        fieldValue := val.Field(i)
        
        validateTag := field.Tag.Get("validate")
        if validateTag == "" {
            continue
        }
        
        // 簡單的驗證邏輯
        if strings.Contains(validateTag, "required") {
            if fieldValue.Kind() == reflect.String && fieldValue.String() == "" {
                errors = append(errors, fmt.Sprintf("%s is required", field.Name))
            }
        }
        
        if strings.Contains(validateTag, "email") {
            email := fieldValue.String()
            if !strings.Contains(email, "@") {
                errors = append(errors, fmt.Sprintf("%s is not a valid email", field.Name))
            }
        }
        
        if strings.Contains(validateTag, "min=0") {
            if fieldValue.Kind() == reflect.Int && fieldValue.Int() < 0 {
                errors = append(errors, fmt.Sprintf("%s must be >= 0", field.Name))
            }
        }
    }
    
    return errors
}

// Calculator 用於方法調用示範
type Calculator struct {
    Value float64
}

func (c *Calculator) Add(x float64) float64 {
    c.Value += x
    return c.Value
}

func (c *Calculator) Multiply(x float64) float64 {
    c.Value *= x
    return c.Value
}

func (c *Calculator) GetValue() float64 {
    return c.Value
}

// 動態方法調用示範
func demonstrateMethodInvocation() {
    calc := &Calculator{Value: 10}
    
    v := reflect.ValueOf(calc)
    t := reflect.TypeOf(calc)
    
    fmt.Printf("    計算器初始值: %.1f\n", calc.Value)
    
    // 動態調用 Add 方法
    addMethod := v.MethodByName("Add")
    if addMethod.IsValid() {
        args := []reflect.Value{reflect.ValueOf(5.0)}
        result := addMethod.Call(args)
        fmt.Printf("    動態調用 Add(5): %.1f\n", result[0].Float())
    }
    
    // 動態調用 Multiply 方法
    multiplyMethod := v.MethodByName("Multiply")
    if multiplyMethod.IsValid() {
        args := []reflect.Value{reflect.ValueOf(2.0)}
        result := multiplyMethod.Call(args)
        fmt.Printf("    動態調用 Multiply(2): %.1f\n", result[0].Float())
    }
    
    // 列出所有可用方法
    fmt.Println("    可用方法:")
    for i := 0; i < t.NumMethod(); i++ {
        method := t.Method(i)
        fmt.Printf("      %s: %s\n", method.Name, method.Type)
    }
}

// 示範 unsafe 套件
func demonstrateUnsafe() {
    fmt.Println("  ⚠️  Unsafe 操作 - 非常危險！")
    
    // 1. 指標算術
    arr := [5]int{10, 20, 30, 40, 50}
    fmt.Printf("    原始陣列: %v\n", arr)
    
    // 獲取第一個元素的指標
    ptr := unsafe.Pointer(&arr[0])
    
    // 透過指標算術存取其他元素
    for i := 0; i < len(arr); i++ {
        // 計算偏移量
        offset := uintptr(i) * unsafe.Sizeof(arr[0])
        elementPtr := (*int)(unsafe.Pointer(uintptr(ptr) + offset))
        fmt.Printf("    索引 %d: %d (透過 unsafe 存取)\n", i, *elementPtr)
    }
    
    // 2. 字串和 []byte 之間的零複製轉換
    str := "Hello, unsafe world!"
    fmt.Printf("    原始字串: %s\n", str)
    
    // 警告：這是不安全的操作！
    bytes := *(*[]byte)(unsafe.Pointer(&str))
    fmt.Printf("    轉換為 bytes: %v\n", bytes)
    
    // 3. 結構欄位偏移量
    p := Person{Name: "測試", Age: 25}
    
    // 獲取欄位偏移量
    nameOffset := unsafe.Offsetof(p.Name)
    ageOffset := unsafe.Offsetof(p.Age)
    
    fmt.Printf("    Person 結構大小: %d bytes\n", unsafe.Sizeof(p))
    fmt.Printf("    Name 欄位偏移量: %d bytes\n", nameOffset)
    fmt.Printf("    Age 欄位偏移量: %d bytes\n", ageOffset)
    
    // 透過偏移量直接存取欄位
    pPtr := unsafe.Pointer(&p)
    agePtr := (*int)(unsafe.Pointer(uintptr(pPtr) + ageOffset))
    fmt.Printf("    透過 unsafe 讀取年齡: %d\n", *agePtr)
    
    fmt.Println("    ⚠️  警告: unsafe 操作可能導致程式崩潰或記憶體洩漏！")
}

// 效能比較
func demonstratePerformanceComparison() {
    fmt.Println("  反射 vs 直接存取效能比較:")
    
    person := Person{Name: "效能測試", Age: 30, Email: "test@example.com"}
    
    // 測試次數
    iterations := 1000000
    
    // 1. 直接存取
    start := time.Now()
    var name string
    for i := 0; i < iterations; i++ {
        name = person.Name
    }
    directTime := time.Since(start)
    
    // 2. 反射存取
    start = time.Now()
    v := reflect.ValueOf(person)
    nameField := v.FieldByName("Name")
    for i := 0; i < iterations; i++ {
        name = nameField.String()
    }
    reflectTime := time.Since(start)
    
    // 3. JSON 序列化比較
    start = time.Now()
    for i := 0; i < 10000; i++ { // 較少次數，因為序列化較慢
        json.Marshal(person)
    }
    jsonTime := time.Since(start)
    
    start = time.Now()
    for i := 0; i < 10000; i++ {
        structToJSON(person)
    }
    reflectJSONTime := time.Since(start)
    
    fmt.Printf("    直接存取 (%d 次): %v\n", iterations, directTime)
    fmt.Printf("    反射存取 (%d 次): %v (慢 %.1fx)\n", 
        iterations, reflectTime, float64(reflectTime)/float64(directTime))
    
    fmt.Printf("    標準 JSON 序列化 (10000 次): %v\n", jsonTime)
    fmt.Printf("    反射 JSON 序列化 (10000 次): %v (慢 %.1fx)\n", 
        reflectJSONTime, float64(reflectJSONTime)/float64(jsonTime))
    
    fmt.Println("\n  效能建議:")
    fmt.Println("    - 反射比直接存取慢 10-100 倍")
    fmt.Println("    - 僅在必要時使用反射（如框架開發）")
    fmt.Println("    - 考慮使用代碼生成代替反射")
    fmt.Println("    - unsafe 提供最高效能，但風險最大")
    
    _ = name // 避免未使用變數警告
}

/*
Cgo 示範（需要 C 編譯器）:

package main

// #include <stdio.h>
// #include <stdlib.h>
// 
// void hello_from_c() {
//     printf("Hello from C!\n");
// }
// 
// int add_numbers(int a, int b) {
//     return a + b;
// }
import "C"

import "fmt"

func demonstrateCgo() {
    // 調用 C 函式
    C.hello_from_c()
    
    // 調用帶參數和回傳值的 C 函式
    result := C.add_numbers(10, 20)
    fmt.Printf("C add_numbers(10, 20) = %d\n", int(result))
    
    // C 字串操作
    cstr := C.CString("Hello from Go")
    defer C.free(unsafe.Pointer(cstr)) // 重要：釋放 C 記憶體
    
    fmt.Printf("C string: %s\n", C.GoString(cstr))
}

編譯 Cgo 程式:
go build -o program main.go

注意事項:
1. Cgo 會顯著增加編譯時間
2. 跨平台支援變得複雜
3. 垃圾回收器無法管理 C 記憶體
4. 調試變得更困難
5. 通常不建議用於效能優化，除非必要
*/